# Stereo Reconstruction with Direct Calibration

In [ ]:
import os
import fluidimage
path_repository = os.path.split(os.path.dirname(fluidimage.__file__))[0]

We first import the class 

In [ ]:
import h5py
from fluidimage.calibration import DirectStereoReconstruction
from fluidimage.calibration.util import get_plane_equation

Get the 2 PIV fields

In [ ]:
def get_piv_field(path):

    try:
        with h5py.File(path, 'r') as f:
            keyspiv = [key for key in f.keys() if key.startswith('piv')]
            keyspiv.sort()
            key = keyspiv[-1]
            X = f[key]['xs'].value
            Y = f[key]['ys'].value
            dx = f[key]['deltaxs_final'].value
            dy = f[key]['deltays_final'].value
    except Exception:
        print(path)
        raise

    return X, Y, dx, dy

pathbase = path_repository + '/image_samples/4th_PIV-Challenge_Case_E'
postfix = '.piv/'
v = 'piv_00001-00002.h5'

path1 = pathbase + '/E_Particle_Images/Camera_01' + postfix + v
path3 = pathbase + '/E_Particle_Images/Camera_03' + postfix + v

Xl, Yl, dxl, dyl = get_piv_field(path1)
Xr, Yr, dxr, dyr = get_piv_field(path3)

Make an instance of the class with the 2 calibration files

In [ ]:
pathcalib1 = pathbase + '/E_Calibration_Images/Camera_01/calib1.npy'
pathcalib3 = pathbase + '/E_Calibration_Images/Camera_03/calib3.npy'
stereo = DirectStereoReconstruction(pathcalib1, pathcalib3)

Define the plane of measurement

In [ ]:
z0 = 0
alpha = 0
beta = 0
a, b, c, d = get_plane_equation(z0, alpha, beta)

Apply calibration on each cameras, the result is given of their respectives planes

In [ ]:
X0, X1, d0cam, d1cam = stereo.project2cam(
    Xl, Yl, dxl, dyl, Xr, Yr, dxr, dyr, a, b, c, d, check=False)

Find the common grid

In [ ]:
X, Y, Z = stereo.find_common_grid(X0, X1, a, b, c, d)

Reconstruct the 3 components of the velocity

In [ ]:
dx, dy, dz, erx, ery, erz = stereo.reconstruction(
    X0, X1, d0cam, d1cam, a, b, c, d, X, Y, check=False)